# Шаги mapreduce для карт
Предварительные наброски кода.

- Deck – колода
- Suit – масть
- Hearts – червы
- Diamonds – бубны
- Clubs – трефы
- Spades – пики
- Jack – валет
- Queen – дама
- King – король
- Ace – туз
- Joker – джокер

## Map
Запишем поток строк в более удобном виде и оставим все значения больше шестерки.

In [ ]:
with open('cards.csv') as f:
    for i, line in enumerate(f):
        suit, value = line.strip().split(',')
        print(suit, value)
        
        if i > 5:
            break

In [ ]:
SUITES = {
    'червы': 'H',
    'пики': 'S',
    'бубны': 'D',
    'трефы': 'C',
}

VALUES = {
    'валет': 'J',
    'дама': 'Q',
    'король': 'K',
    'туз': 'A',
}

In [ ]:
with open('cards.csv') as f:
    for i, line in enumerate(f):
        suit, value = line.strip().split(',')
        
        suit = SUITES[suit]
        
        if value in VALUES:
            value = VALUES[value]
        
        print(f'{suit},{value},1')
        
        if i > 5:
            break

In [ ]:
with open('cards_after_mapper.csv', 'w') as f_mapper:
    with open('cards.csv') as f:
        for i, line in enumerate(f):
            suit, value = line.strip().split(',')

            suit = SUITES[suit]

            if value in VALUES:
                value = VALUES[value]

            if value.isdigit() and int(value) >= 6:
                f_mapper.write(f'{suit},{value},1\n')

## Shuffle
```
cat cards_after_mapper.csv | sort > cards_sorted.csv
```

## Reduce
Сначала посчитаем количество мастей карт

In [ ]:
with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        print(suit, value, one)
        
        if i > 5:
            break

In [ ]:
previous_suit = None
suit_count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit:
                suit_count += one
            else:
                print(previous_suit, suit_count)
                suit_count = one
                previous_suit = suit
        else:
            previous_suit = suit
            suit_count = one
        
print(previous_suit, suit_count)

Уберем дубликат строчки
```python
previous_suit = suit
```

In [ ]:
previous_suit = None
suit_count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit:
                suit_count += one
            else:
                print(previous_suit, suit_count)
                suit_count = one   
        else:
            suit_count = one
        
        previous_suit = suit
        
print(previous_suit, suit_count)

## Reduce v2
Посчитаем статистику для пар масть + значение

In [ ]:
previous_suit = None
previous_value = None

count = 0

with open('cards_sorted.csv') as f:
    for i, line in enumerate(f):
        suit, value, one = line.strip().split(',')
        one = int(one)
        
        if previous_suit:
            if previous_suit == suit and previous_value == value:
                count += one
            else:
                print(previous_suit, previous_value, count)
                
                previous_suit = suit
                previous_value = value
                count = one
                
        else:
            previous_suit = suit
            previous_value = value
            count = one
            
print(previous_suit, previous_value, count)